<a href="https://colab.research.google.com/github/akitim/comptech-winter-school-2022_covid19-prediction/blob/main/prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from pathlib import Path

In [20]:
!git clone https://github.com/comptech-winter-school/covid19-prediction

fatal: destination path 'covid19-prediction' already exists and is not an empty directory.


In [21]:
path = "covid19-prediction"
all_files = Path(path).glob("data*.csv") 

df = pd.concat(map(pd.read_csv, all_files));

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
# path = "COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"
# all_files = Path(path).glob("*.csv") 

# df = pd.concat(map(pd.read_csv, all_files))

In [15]:
df.shape

(2637010, 22)

In [16]:
df.head()

,Unnamed: 0,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio,Incident_Rate,Case_Fatality_Ratio,Province/State,Country/Region,Last Update,Latitude,Longitude
0,485,22027.0,Claiborne,Louisiana,US,2020-05-24 02:32:43,32.823085,-92.995269,58.0,10.0,0.0,48.0,"Claiborne, Louisiana, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,486,28021.0,Claiborne,Mississippi,US,2020-05-24 02:32:43,31.970617,-90.907022,57.0,1.0,0.0,56.0,"Claiborne, Mississippi, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,487,47025.0,Claiborne,Tennessee,US,2020-05-24 02:32:43,36.483191,-83.659473,8.0,0.0,0.0,8.0,"Claiborne, Tennessee, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,488,53009.0,Clallam,Washington,US,2020-05-24 02:32:43,48.047546,-123.922632,25.0,0.0,0.0,25.0,"Clallam, Washington, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,489,26035.0,Clare,Michigan,US,2020-05-24 02:32:43,43.987942,-84.848199,17.0,2.0,0.0,15.0,"Clare, Michigan, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def merge_columns(f1, f2, df):
    d = df[[f1, f2]].copy().reset_index()
    d.loc[~d[f2].isna(), f1] = df.loc[~d[f2].isna(), f2]
    result = d[f1]
    result[result.isna()] = df.loc[result.isna(), f2]
    return result

In [23]:
def remove_small_countries(df):
    countries = df.groupby("Country_Region").last().query('Confirmed > 100000').index.to_list()
    return df[df.Country_Region.isin(countries)]

In [24]:
def report_shape(df):
    print(df.shape)
    return df

In [8]:
(df
 .assign(Last_Update = merge_columns("Last_Update", "Last Update", df))
 .assign(Country_Region = merge_columns("Country_Region", "Country/Region", df))
 .assign(Last_Update = pd.to_datetime(df["Last_Update"]).dt.date)
 .loc[:, ["Country_Region", "Last_Update", "Confirmed"]]
 .dropna()
 .groupby(["Country_Region", "Last_Update"])
 .sum()
 .reset_index()
 .set_index("Last_Update")
 .sort_index()
 .pipe(remove_small_countries)
 .pipe(report_shape)
 .to_parquet("tmp/data.parquet")
)

(80185, 2)


In [25]:
def df_nans_calc(df):
    valid_se = (~df.isna()).sum(axis=0)
    df_nans = pd.concat([valid_se, df.shape[0] - valid_se], axis=1)
    df_nans.columns = ['valid', 'NaNs']
    return df_nans

df_nans_calc(df)

,valid,NaNs
Unnamed: 0,2637010,0
FIPS,2163669,473341
Admin2,2167116,469894
Province_State,2510659,126351
Country_Region,2628068,8942
Last_Update,2628068,8942
Lat,2571664,65346
Long_,2571664,65346
Confirmed,2636982,28
Deaths,2636576,434


In [32]:
merge_columns('Country_Region', 'Country/Region', df).isna().sum()
pd.isna(df['Country/Region'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0         True
1         True
2         True
3         True
4         True
          ... 
527397    True
527398    True
527399    True
527400    True
527401    True
Name: Country/Region, Length: 2637010, dtype: bool